In [8]:
from etl import *
import pandas as pd
from mongodb import Init, clear_indexes
from datetime import datetime
from functools import lru_cache
from dicts import *

clear_indexes("tracings")

await Init()

# config
PERIOD = "Oct"
YEAR = "2022"
SALES_COL = "TotSales" + PERIOD
KEY_BASE = f"{PERIOD}_{YEAR}_"



@lru_cache(maxsize=1000)
def cached_find_rep(item: str, state: str, postal: str, cardinal_division: str = None) -> str:     
     cardinal_rep = None
     
     rep = FMcustomkits.get(item, "Rep40")

     if rep == "Rep40":
          rep = state_dict.get(state, "Rep40")
          
     if rep == "Rep40":
          rep = zip_dict.get(postal, "Rep40")

     if cardinal_division is not None and rep != "Rep40":
          cardinal_rep = cardinal_dict.get(cardinal_division, "Rep40")          

          if cardinal_rep == rep:
               return cardinal_rep
          
     return rep

@lru_cache(maxsize=1000)
def cached_find_commission_rate(item: str) -> float:     
     return commission_rates.get(item, 0.04)

In [ ]:
# house and henryschein - non traced
df = pd.read_excel(house, sheet_name=sheet)

df["PartNum"] = df["PartNum"].astype(str)

df["SalesRep"] = df["SalesRep"].astype(str)
df["SalesRep"] = df["SalesRep"].str.replace(r".0","")


In [ ]:
key = KEY_BASE + "Henryschein"

henryschein = df.copy()
henryschein.dropna(subset=["hsrep"], inplace=True)
henryschein["DistCode"] = henryschein["DistCode"].astype(str)
henryschein = henryschein[(henryschein["DistCode"] == "2091") & (henryschein[SALES_COL] != 0)]

henryschein = henryschein[["PartNum", "Name", "hsrep", "pct", SALES_COL]]

await Tracing.find(Tracing.key == key).delete()

for _, x in henryschein.iterrows():
    try:
        t = Tracing(
            rep=x["hsrep"],
            item=x["PartNum"],
            sale=x[SALES_COL],
            commission=x["pct"],
            date_of_sale=datetime(2022, 10, 1, 0, 0, 0),
            key=key,
            name=x["Name"],
            address="",
            address2="",
            city="",
            state="",
            zip="",
            country="",
            created_at=datetime.now(),
            updated_at=datetime.now(),
        )

        await Tracing.insert_one(t)
    except:
        print(t)
    


In [ ]:
key = KEY_BASE + "Busse"

busse = df[(df["SalesRep"] != "nan") & (df["SalesRep"] != "") & (df[SALES_COL] != 0)].copy()

busse["SalesRep"] = busse["SalesRep"].replace("14","41").replace("22","41").replace("23","41")
busse["SalesRep"] = "Rep" + busse["SalesRep"]

busse["DistCode"] = busse["DistCode"].astype(str)
busse = busse[~busse["DistCode"].isin([str(x) for x in [1300, 1402, 1404, 1070, 2091, 2614, 1716, 3418, 2450, 6820]])]

dists = ["2084"]
reps = ["Rep13", "Rep28", "Rep23", "Rep24", "Rep41"]

for dist in dists:
    for rep in reps:
        busse.loc[(busse["DistCode"] == dist) & (busse["SalesRep"] == rep), SALES_COL] = 0.0

busse.loc[(busse["DistCode"] == "2233") & (busse["SalesRep"] == "Rep1") & (busse["PartNum"] == "6820"), SALES_COL] = 0.0

busse = busse[busse[SALES_COL] != 0]

busse[(busse["DistCode"] == "2084") & (busse["SalesRep"] == "Rep13")]

busse = busse[["PartNum", "Name", "SalesRep", SALES_COL]]

await Tracing.find(Tracing.key == key).delete()

for _, x in busse.iterrows():
    try:
        t = Tracing(
            rep=x["SalesRep"],
            item=x["PartNum"],
            sale=x[SALES_COL],
            commission=0.04,
            date_of_sale=datetime(2022, 10, 1, 0, 0, 0),
            key=key,
            name=x["Name"],
            address="",
            address2="",
            city="",
            state="",
            zip="",
            country="",        
            created_at=datetime.now(),
            updated_at=datetime.now(),
        )
        await Tracing.insert_one(t)
    except Exception as e:
        print(t)

# insert tracings to db

await Tracing.insert_many(busse_tracings)


In [9]:
key = KEY_BASE + "Cardinal"
df = pd.read_excel(cardinal, sheet_name=sheet, parse_dates=["date"])

def find_rep(row: pd.Series):
     item = row["item"]
     state = row["state"]
     postal = row["l"]
     cardinal_division = row["SalesRep"]

     return cached_find_rep(item, state, postal, cardinal_division)

df["commission"] = df["item"].apply(cached_find_commission_rate)
df["rep"] = df.apply(find_rep, axis=1)    

await Tracing.find(Tracing.key == key).delete()

for _, x in df.iterrows():     
     try:
          t = Tracing(
               rep=x["rep"],
               item=x["item"],
               sale=x["sale"],
               commission=x["commission"],
               date_of_sale=x["date"],
               key=key,
               name=x["g"],
               address=x["h"],
               address2="",
               city=x["i"],
               state=x["state"],
               zip=x["zip"],
               country="",
               created_at=datetime.now(),
               updated_at=datetime.now(),          
          )
          await Tracing.insert_one(t)
          
     except:         
          print(t)



id=ObjectId('637553737fef11a9b235e8c2') revision_id=None rep=<Rep.REP35: 'Rep35'> item='717' commission=0.04 sale=17.24 date_of_sale=Timestamp('2022-10-14 00:00:00') key='Oct_2022_Cardinal' name='NORTHWEST SPECIALTY HOSPITAL' address='1593 E POLSTON AVE' address2='' city='POST FALLS' state='ID' zip='838545326' country='' created_at=datetime.datetime(2022, 11, 16, 16, 17, 39, 276876) updated_at=datetime.datetime(2022, 11, 16, 16, 17, 39, 276876)
id=ObjectId('637553737fef11a9b235e8c2') revision_id=None rep=<Rep.REP35: 'Rep35'> item='717' commission=0.04 sale=17.24 date_of_sale=Timestamp('2022-10-14 00:00:00') key='Oct_2022_Cardinal' name='NORTHWEST SPECIALTY HOSPITAL' address='1593 E POLSTON AVE' address2='' city='POST FALLS' state='ID' zip='838545326' country='' created_at=datetime.datetime(2022, 11, 16, 16, 17, 39, 276876) updated_at=datetime.datetime(2022, 11, 16, 16, 17, 39, 276876)
id=ObjectId('637553757fef11a9b235f19e') revision_id=None rep=<Rep.REP1: 'Rep1'> item='791' commission=

In [ ]:
key = KEY_BASE + "Medline"

df = pd.read_excel(medline, parse_dates=["InvoiceDat"])

def find_rep(row: pd.Series):
     item = row["VendorItm"]
     state = row["CustState"]
     postal = row["CustZipcod"][:3]
     division = None

     return cached_find_rep(item, state, postal, division)

df["commission"] = df["VendorItm"].apply(cached_find_commission_rate)
df["rep"] = df.apply(find_rep, axis=1)

await Tracing.find(Tracing.key == key).delete()

for _, x in df.iterrows():
     try:
          t = Tracing(
               rep=x["rep"],
               item=x["VendorItm"],
               sale=(x["AcqCost"] * x["Quantity"]) - x["RebateAmt"],
               commission=x["commission"],
               date_of_sale=x["InvoiceDat"] if x["InvoiceDat"] >= datetime(2022, 10, 1, 0, 0, 0) else datetime(2022, 10, 1, 0, 0, 0),
               key=key,
               name=x["CustName"],
               address=x["CustStreet"],
               address2="",
               city=x["CustCity"],
               state=x["CustState"],
               zip=x["CustZipcod"],
               country="",
               created_at=datetime.now(),
               updated_at=datetime.now(),          
          )
          await Tracing.insert_one(t)
          
     except:         
          print(t)



